**Reference**: 
- [x] [Amazon Bedrock](https://community.aws/content/2hW7srTWRb5idHjY4I8WP5fQFRf/build-a-tool-use-based-agent-loop-with-amazon-bedrock)

In [22]:
import json
import math
from pprint import pprint

from gen_ai_hub.proxy.native.amazon.clients import Session

model_name = 'anthropic--claude-3-opus'
bedrock = Session().client(model_name=model_name)

In [23]:
tool_list = [
    {
        "toolSpec": {
            "name": "cosine",
            "description": "Calculate the cosine of x",
            "inputSchema": {
                "json": {
                    "type": "object", 
                    "properties": {
                        "x": {
                            "type": "number", 
                            "description": "The number to pass to the function"
                        }
                    },
                    "required": ["x"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "sine",
            "description": "Calculate the sine of x",
            "inputSchema": {
                "json": {
                    "type": "object", 
                    "properties": {
                        "x": {
                            "type": "number", 
                            "description": "The number to pass to the function"
                        }
                    },
                    "required": ["x"]
                }
            }
        }
    }, 
    {
        "toolSpec": {
            "name": "addition", 
            "description": "Add the result of x and y", 
            "inputSchema":{
                "json": {
                    "type": "object",
                    "properties": {
                        "x": {
                            "type": "number", 
                            "description": "The number to pass to the function"
                        }, 
                        "y": {
                            "type": "number", 
                            "description": "The number to pass to the function"
                        }
                    },
                    "required": ["x", "y"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "get_numerical",
            "description":  "Extracts only the numerical value from a given answer, ignoring explanations or other text.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                    "answer": {
                            "type": "string",
                            "description": "The input string that may contain a number along with other text. This function extracts the numerical value."
                        }
                    },
                    "required": ["answer"]
                }
            }
        }
    },
]        

In [24]:
def get_tool_use_result(tool_use_block):
    tool_use_name = tool_use_block['name']
    tool_use_input = tool_use_block['input']
    
    if tool_use_name == "cosine":
        x = tool_use_input['x']
        x = float(x)
        return math.cos(x)
    elif tool_use_name == "sine":
        x = tool_use_input['x']
        x = float(x)
        return math.sin(x)
    elif tool_use_name == "addition":
        x = tool_use_input['x']
        y = tool_use_input["y"]
        x = float(x)
        y = float(y)
        return x + y
    elif tool_use_name == "get_numerical":
        answer = tool_use_input['answer']
        return float(answer)

In [25]:
def handle_response(response_message):
    response_content_blocks = response_message['content']
    follow_up_content_blocks = []
    for content_block in response_content_blocks:
        if 'toolUse' in content_block:
            tool_use_block = content_block['toolUse']
            tool_use_id = tool_use_block["toolUseId"]
            tool_result_value = get_tool_use_result(tool_use_block)
            if tool_result_value is not None:
                follow_up_content_blocks.append(
                    {"toolResult": {
                            "toolUseId": tool_use_id,
                            "content": [
                                {"json": {"result": tool_result_value} }
                            ]
                        }
                    }
                )
    if len(follow_up_content_blocks) > 0:
        follow_up_messages = {
            "role": "user", 
            "content": follow_up_content_blocks, 
        }

        return follow_up_messages

In [26]:
def call_bedrock(message_list, tool_list):
    bedrock = Session().client(model_name=model_name)
    response = bedrock.converse(
        messages = message_list, 
        inferenceConfig = {
            "maxTokens": 4096, 
            "temperature": 0.0, 
        },
        toolConfig={
            "tools": tool_list
        }
    )
    return response

In [27]:
def run_loop(prompt, tool_list):
    MAX_LOOPS = 5
    loop_count = 0
    continue_loop = True

    message_list = []
    initial_message = {
        "role": "user", 
        "content": [
            {"text": prompt}
        ]
    }
    message_list.append(initial_message)

    while continue_loop:
        pprint(message_list)
        print('-'*10)
        response = call_bedrock(message_list, tool_list)
        response_message = response['output']['message']
        message_list.append(response_message)

        loop_count = loop_count + 1
        if loop_count > MAX_LOOPS:
            print('hit loop limit')
            break

        follow_up_message = handle_response(response_message)
        if follow_up_message is None:
            continue_loop = False
        else:
            message_list.append(follow_up_message)

    return message_list

In [28]:
message = run_loop("What is cos(1000) + sin(8000)? Please do it step by step.", tool_list)

[{'content': [{'text': 'What is cos(1000) + sin(8000)? Please do it step by '
                       'step.'}],
  'role': 'user'}]
----------
[{'content': [{'text': 'What is cos(1000) + sin(8000)? Please do it step by '
                       'step.'}],
  'role': 'user'},
 {'content': [{'text': '<thinking>\n'
                       'To answer this question, we will need to use the '
                       'cosine and sine functions, and then add the results '
                       'together using the addition function.\n'
                       '\n'
                       'The cosine function requires the parameter x, which is '
                       '1000 in this case. We have that value provided '
                       'directly in the question.\n'
                       '\n'
                       'The sine function also requires the parameter x, which '
                       'is 8000 in this case. We also have that value provided '
                       'directly.\n'
         

In [30]:
pprint(message)

[{'content': [{'text': 'What is cos(1000) + sin(8000)? Please do it step by '
                       'step.'}],
  'role': 'user'},
 {'content': [{'text': '<thinking>\n'
                       'To answer this question, we will need to use the '
                       'cosine and sine functions, and then add the results '
                       'together using the addition function.\n'
                       '\n'
                       'The cosine function requires the parameter x, which is '
                       '1000 in this case. We have that value provided '
                       'directly in the question.\n'
                       '\n'
                       'The sine function also requires the parameter x, which '
                       'is 8000 in this case. We also have that value provided '
                       'directly.\n'
                       '\n'
                       'Once we have the results of cosine and sine, we can '
                       'pass those as the x a